In [1]:
# Dependencies and Setup
import os
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import requests
import json

In [2]:
#read the data sets from resources directory
csv_path = "salary_prediction.csv"
states_train_data = pd.read_csv(csv_path)
states_train_data

,age,workclass,education,marital,occupation,race,sex,workhour,location,above/below50K
0,39,State-gov,Bachelors,Never-married,Adm-clerical,White,Male,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,White,Male,13,United-States,<=50K
2,38,Private,HS-grad,Divorced,Handlers-cleaners,White,Male,40,United-States,<=50K
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Black,Male,40,United-States,<=50K
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Black,Female,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,Assoc-acdm,Married-civ-spouse,Tech-support,White,Female,38,United-States,<=50K
32557,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,White,Male,40,United-States,>50K
32558,58,Private,HS-grad,Widowed,Adm-clerical,White,Female,40,United-States,<=50K
32559,22,Private,HS-grad,Never-married,Adm-clerical,White,Male,20,United-States,<=50K


In [3]:
states_train_data.dtypes

age                int64
workclass         object
education         object
marital           object
occupation        object
race              object
sex               object
workhour           int64
location          object
above/below50K    object
dtype: object

In [4]:
#finding the data that has '?' mark 
states_train_data.isin([' ?']).sum(axis=0)

age                  0
workclass         1836
education            0
marital              0
occupation        1843
race                 0
sex                  0
workhour             0
location           583
above/below50K       0
dtype: int64

In [5]:
# code will replace the special character to nan and then drop the columns 
states_train_data['location'] = states_train_data['location'].replace(' ?',np.nan)
states_train_data['workclass'] = states_train_data['workclass'].replace(' ?',np.nan)
states_train_data['occupation'] = states_train_data['occupation'].replace(' ?',np.nan)
#dropping the NaN rows now 
states_train_data.dropna(how='any',inplace=True)

In [6]:
#double check if there is '?' mark 
states_train_data.isin([' ?']).sum(axis=0)

age               0
workclass         0
education         0
marital           0
occupation        0
race              0
sex               0
workhour          0
location          0
above/below50K    0
dtype: int64

In [7]:
# states_train_data = states_train_data.replace({"Private": "private"})
# states_train_data.columns

In [8]:
states_train_data['workclass']=states_train_data['workclass'].str.split('-',expand=True)
states_train_data

,age,workclass,education,marital,occupation,race,sex,workhour,location,above/below50K
0,39,State,Bachelors,Never-married,Adm-clerical,White,Male,40,United-States,<=50K
1,50,Self,Bachelors,Married-civ-spouse,Exec-managerial,White,Male,13,United-States,<=50K
2,38,Private,HS-grad,Divorced,Handlers-cleaners,White,Male,40,United-States,<=50K
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Black,Male,40,United-States,<=50K
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Black,Female,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,Assoc-acdm,Married-civ-spouse,Tech-support,White,Female,38,United-States,<=50K
32557,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,White,Male,40,United-States,>50K
32558,58,Private,HS-grad,Widowed,Adm-clerical,White,Female,40,United-States,<=50K
32559,22,Private,HS-grad,Never-married,Adm-clerical,White,Male,20,United-States,<=50K


In [9]:
states_train_data['workclass'].value_counts()

 Private    22286
 Self        3573
 Local       2067
 State       1279
 Federal      943
 Without       14
Name: workclass, dtype: int64

In [10]:
states_train_data['education']=states_train_data['education'].str.split('-',expand=True)
states_train_data['education']

0         Bachelors
1         Bachelors
2                HS
3              11th
4         Bachelors
            ...    
32556         Assoc
32557            HS
32558            HS
32559            HS
32560            HS
Name: education, Length: 30162, dtype: object

In [11]:
states_train_data['marital']=states_train_data['marital'].str.split('-',expand=True)
states_train_data

,age,workclass,education,marital,occupation,race,sex,workhour,location,above/below50K
0,39,State,Bachelors,Never,Adm-clerical,White,Male,40,United-States,<=50K
1,50,Self,Bachelors,Married,Exec-managerial,White,Male,13,United-States,<=50K
2,38,Private,HS,Divorced,Handlers-cleaners,White,Male,40,United-States,<=50K
3,53,Private,11th,Married,Handlers-cleaners,Black,Male,40,United-States,<=50K
4,28,Private,Bachelors,Married,Prof-specialty,Black,Female,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,Assoc,Married,Tech-support,White,Female,38,United-States,<=50K
32557,40,Private,HS,Married,Machine-op-inspct,White,Male,40,United-States,>50K
32558,58,Private,HS,Widowed,Adm-clerical,White,Female,40,United-States,<=50K
32559,22,Private,HS,Never,Adm-clerical,White,Male,20,United-States,<=50K


In [12]:
states_train_data['location']=states_train_data['location'].str.replace('-','_')
states_train_data

,age,workclass,education,marital,occupation,race,sex,workhour,location,above/below50K
0,39,State,Bachelors,Never,Adm-clerical,White,Male,40,United_States,<=50K
1,50,Self,Bachelors,Married,Exec-managerial,White,Male,13,United_States,<=50K
2,38,Private,HS,Divorced,Handlers-cleaners,White,Male,40,United_States,<=50K
3,53,Private,11th,Married,Handlers-cleaners,Black,Male,40,United_States,<=50K
4,28,Private,Bachelors,Married,Prof-specialty,Black,Female,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,Assoc,Married,Tech-support,White,Female,38,United_States,<=50K
32557,40,Private,HS,Married,Machine-op-inspct,White,Male,40,United_States,>50K
32558,58,Private,HS,Widowed,Adm-clerical,White,Female,40,United_States,<=50K
32559,22,Private,HS,Never,Adm-clerical,White,Male,20,United_States,<=50K


In [13]:
states_train_data['occupation']=states_train_data['occupation'].str.replace('-','_')
states_train_data

,age,workclass,education,marital,occupation,race,sex,workhour,location,above/below50K
0,39,State,Bachelors,Never,Adm_clerical,White,Male,40,United_States,<=50K
1,50,Self,Bachelors,Married,Exec_managerial,White,Male,13,United_States,<=50K
2,38,Private,HS,Divorced,Handlers_cleaners,White,Male,40,United_States,<=50K
3,53,Private,11th,Married,Handlers_cleaners,Black,Male,40,United_States,<=50K
4,28,Private,Bachelors,Married,Prof_specialty,Black,Female,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,Assoc,Married,Tech_support,White,Female,38,United_States,<=50K
32557,40,Private,HS,Married,Machine_op_inspct,White,Male,40,United_States,>50K
32558,58,Private,HS,Widowed,Adm_clerical,White,Female,40,United_States,<=50K
32559,22,Private,HS,Never,Adm_clerical,White,Male,20,United_States,<=50K


In [14]:
# states_train_data.loc[states_train_data.workclass=='State', 'workclass'] = states_train_data.workclass.str.replace('e', 'E!')


In [15]:
states_train_data

,age,workclass,education,marital,occupation,race,sex,workhour,location,above/below50K
0,39,State,Bachelors,Never,Adm_clerical,White,Male,40,United_States,<=50K
1,50,Self,Bachelors,Married,Exec_managerial,White,Male,13,United_States,<=50K
2,38,Private,HS,Divorced,Handlers_cleaners,White,Male,40,United_States,<=50K
3,53,Private,11th,Married,Handlers_cleaners,Black,Male,40,United_States,<=50K
4,28,Private,Bachelors,Married,Prof_specialty,Black,Female,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,Assoc,Married,Tech_support,White,Female,38,United_States,<=50K
32557,40,Private,HS,Married,Machine_op_inspct,White,Male,40,United_States,>50K
32558,58,Private,HS,Widowed,Adm_clerical,White,Female,40,United_States,<=50K
32559,22,Private,HS,Never,Adm_clerical,White,Male,20,United_States,<=50K


In [16]:
replace_workclass = {'State':'Government', 'Federal':'Government', 'Local':'Government'}
states_train_data = states_train_data.replace({"workclass": replace_workclass},regex=True)
states_train_data

,age,workclass,education,marital,occupation,race,sex,workhour,location,above/below50K
0,39,Government,Bachelors,Never,Adm_clerical,White,Male,40,United_States,<=50K
1,50,Self,Bachelors,Married,Exec_managerial,White,Male,13,United_States,<=50K
2,38,Private,HS,Divorced,Handlers_cleaners,White,Male,40,United_States,<=50K
3,53,Private,11th,Married,Handlers_cleaners,Black,Male,40,United_States,<=50K
4,28,Private,Bachelors,Married,Prof_specialty,Black,Female,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,Assoc,Married,Tech_support,White,Female,38,United_States,<=50K
32557,40,Private,HS,Married,Machine_op_inspct,White,Male,40,United_States,>50K
32558,58,Private,HS,Widowed,Adm_clerical,White,Female,40,United_States,<=50K
32559,22,Private,HS,Never,Adm_clerical,White,Male,20,United_States,<=50K


In [17]:
# states_train_data_work = states_train_data['workclass'].replace(to_replace = ['State','Federal','Local'], value = 'Government',regex=True)


In [18]:
replace_edu = {'Assoc' : 'Associate_Degree', 'Some' : 'Bachelors','HS' : 'High_School_Degree', '1st' : 'Not_Graduated','11th' : 'Not_Graduated','4th' : 'Not_Graduated','12th' : 'Not_Graduated','5th' : 'Not_Graduated','10th' : 'Not_Graduated', '7th' : 'Not_Graduated','Masters':'Graduate','Doctorate':'Graduate','Prof':'Graduate', '9th' : 'Not_Graduated'} 

In [19]:
states_train_data = states_train_data.replace({"education": replace_edu},regex=True)
states_train_data['education'].value_counts()
# state_df = state_df.replace({"state": replace_values})    

 Bachelors             11722
 High_School_Degree     9840
 Not_Graduated          3696
 Graduate               2544
 Associate_Degree       2315
 Preschool                45
Name: education, dtype: int64

In [20]:
replace_marital = {'Never' : 'Single'} 
states_train_data = states_train_data.replace({"marital": replace_marital},regex=True)
states_train_data['marital'].value_counts()

 Married      14456
 Single        9726
 Divorced      4214
 Separated      939
 Widowed        827
Name: marital, dtype: int64

In [21]:
states_train_data['occupation'].value_counts()

 Prof_specialty       4038
 Craft_repair         4030
 Exec_managerial      3992
 Adm_clerical         3721
 Sales                3584
 Other_service        3212
 Machine_op_inspct    1966
 Transport_moving     1572
 Handlers_cleaners    1350
 Farming_fishing       989
 Tech_support          912
 Protective_serv       644
 Priv_house_serv       143
 Armed_Forces            9
Name: occupation, dtype: int64

In [22]:
replace_occupation = {'Prof_specialty' : 'Managerial', 'Craft_repair':'Service', 'Exec_managerial':'Managerial', 'Adm_clerical':'Clerical', 'Other_service':'Serivce', 'Machine_op_inspct': 'Technical', 'Transport_moving':'Technical', 'Handlers_cleaners':'Service', 'Farming_fishing': 'Agricultural', 'Tech_support':'Technical', 'Protective_serv':'Technical', 'Priv_house_serv':'Service', 'Armed_Forces':'Service'} 
states_train_data = states_train_data.replace({"occupation": replace_occupation},regex=True)
states_train_data['occupation'].value_counts()

 Managerial      8030
 Service         5532
 Technical       5094
 Clerical        3721
 Sales           3584
 Serivce         3212
 Agricultural     989
Name: occupation, dtype: int64

In [23]:
states_train_data['location'].value_counts()

 United_States                 27504
 Mexico                          610
 Philippines                     188
 Germany                         128
 Puerto_Rico                     109
 Canada                          107
 El_Salvador                     100
 India                           100
 Cuba                             92
 England                          86
 Jamaica                          80
 South                            71
 Italy                            68
 China                            68
 Dominican_Republic               67
 Vietnam                          64
 Guatemala                        63
 Japan                            59
 Columbia                         56
 Poland                           56
 Taiwan                           42
 Iran                             42
 Haiti                            42
 Portugal                         34
 Nicaragua                        33
 Peru                             30
 Greece                           29
 

In [25]:
replace_countries = {'United_States' : 'North_America', 'Canada':'North_America', 'Mexico':'South_America', 'Philippines':'Asia', 'Germany':'Europe', 'Puerto_Rico':'South_America', 'India': 'Asia', 'El_Salvador':'South_America', 'Cuba':'South_America', 'England':'Europe', 'Jamaica': 'South_America', 'South':'Asia', 'Italy':'Europe', 'China':'Asia', 'Dominican_Republic':'South_America', 'Vietnam':'Asia', 'Guatemala':'South_America', 'Japan':'Asia', 'Poland':'Europe', 'Columbia':'South_America', 'Haiti':'South_America', 'Iran':'Asia', 'Taiwan':'Asia', 'Nicaragua':'South_America','Peru':'South_America','Greece':'Europe','France':'Europe', 'Ecuador':'South_America', 'Ireland':'Europe','Hong':'Asia', 'Trinadad&Tobago':'South_America','Cambodia':'Asia', 'Laos':'Asia', 'Yugoslavia':'Europe', 'Outlying_US':'North_America', 'Hungary':'Europe','Scotland':'Europe', 'Holand_Netherlands':'Europe', 'Honduras':'South_America', 'Thailand':'Asia', 'Portugal':'Europe'}
states_train_data = states_train_data.replace({"location": replace_countries},regex=True)
states_train_data['location'].value_counts()

 North_America                   27611
 Asia_America                     1339
 Asia                              705
 Europe                            493
 North_America(Guam_USVI_etc)       14
Name: location, dtype: int64

In [26]:
replace_race = {'Asian-Pac-Islander' : 'Asian', 'Amer-Indian-Eskimo':'Aboriginal'}
states_train_data = states_train_data.replace({"race": replace_race},regex=True)
states_train_data['race'].value_counts()

 White         25933
 Black          2817
 Asian           895
 Aboriginal      286
 Other           231
Name: race, dtype: int64

In [ ]:
# states_train_data_college = states_train_data['education'].replace(to_replace = ['Some'], value = 'Uni_Not_Graduated',regex=True)
# states_train_data_college

In [ ]:
# states_train_data_result = states_train_data.replace(to_replace = ['State','Federal','Local'], value = 'Government',regex=True)
# states_train_data_result

# df.loc[lambda df: df['shield'] == 8]

In [ ]:
# result = sr.replace(to_replace = ['New York', 'Rio'], value = ['London', 'Brisbane'])

In [ ]:
# states_train_data.to_csv('salary_cleaned.csv', index=False, header=True)
# states_train_data.to_csv('salary_test_cleaned.csv', index=False,header=True)

In [ ]:
# #data_cleaning
# states_train_data = states_train_data.drop(columns=['Unnamed: 0','Unnamed: 3','Unnamed: 5','Unnamed: 8','Unnamed: 11','Unnamed: 12'] )
# states_train_data

In [ ]:
# labeled_train_df = states_train_data.rename(columns={'1':'Age', '2':'Sector', '4':'Education', '6': 'Marital_Status', '7':'Occupation', '9': 'Race', '10': 'Sex', '14':'Location','15':'Above/Below 50K'})
# labeled_train_df

In [30]:
states_train_data.to_csv("salary_prediction_ML.csv",index=False, header=True)
states_train_data.to_csv("salary_prediction_ML_test.csv",index=False, header=True)